In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Oct  5 19:45:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install patool
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install flask-bootstrap
!pip install faiss-gpu
!pip install torchcam
!pip install torchvision

  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.


  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchcam 0.3.2 requires torch<2.0.0,>=1.7.0, but you have torch 2.0.1 which is incompatible.


In [ ]:
!ngrok authtoken 29ICcwUSREbizrNLwbyiamd6RjX_C3jsNzyb72RRytkrqgJo


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install --upgrade Pillow
!pip install PIL
!pip install image

In [ ]:
import matplotlib.pyplot as plt
import os
import glob
from itertools import chain
import random
import zipfile
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from PIL import Image ,ImageOps
from sklearn.model_selection import train_test_split
import cv2
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision import datasets, transforms, models
import faiss
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision import models
from torchcam.methods import LayerCAM
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask
from torchvision import models

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)

    # Distances in embedding space is calculated in euclidean
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [ ]:
class TripletData(Dataset):
    def __init__(self, path, transforms, split,cats):
        self.path = path
        self.split = split    # train or valid
        self.cats = cats     # number of categories
        self.transforms = transforms

    def __getitem__(self, idx):
        # our positive class for the triplet
        idx = str(idx%self.cats + 1)

        # choosing our pair of positive images (im1, im2)
        positives = os.listdir(os.path.join(self.path, idx))
        im1, im2 = random.sample(positives, 2)

        # choosing a negative class and negative image (im3)
        negative_cats = [str(x+1) for x in range(self.cats)]
        negative_cats.remove(idx)
        negative_cat = str(random.choice(negative_cats))
        negatives = os.listdir(os.path.join(self.path, negative_cat))
        im3 = random.choice(negatives)

        im1,im2,im3 = os.path.join(self.path, idx, im1), os.path.join(self.path, idx, im2), os.path.join(self.path, negative_cat, im3)
        im1 = Image.open(im1).convert("RGB")
        im2=Image.open(im2).convert("RGB")
        im3=Image.open(im3).convert("RGB")
        im1 = self.transforms(im1)
        im2 = self.transforms(im2)
        im3 = self.transforms(im3)

        return [im1, im2, im3]

    # we'll put some value that we want since there can be far too many triplets possible
    # multiples of the number of images/ number of categories is a good choice
    def __len__(self):
        return self.cats*8


# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((220,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

val_transforms = transforms.Compose([
    transforms.Resize((220, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
def get_optimizer(model,option,lr):
    if option == "Adam":
       return optim.Adam(model.parameters(), lr=lr)
    elif option == "AdamaX":
        return optim.Adamax(model.parameters(), lr=lr)
    elif option == "AdamW":
        return optim.AdamW(model.parameters(), lr=lr)
    elif option=="SGD":
        return optim.SGD(model.parameters(), lr=lr)
    elif option=="ASGD":
        return optim.ASGD(model.parameters(), lr=lr)

    else:
        return None


In [ ]:
def get_model(option):
    if option == "resnet18":
       return models.resnet18()
    elif option =="alexnet":
        return models.alexnet()
    elif option == "vgg19":
        return models.vgg19()
    elif option=="resnet50":
        return models.resnet50()
    elif option=="googlenet":
        return models.googlenet()
    elif option=="squeezenet1_1":
        return models.squeezenet1_1()
    else:
        return None


In [ ]:
def get_model1(option):
    if option == "resnet18":
       return models.resnet18(pretrained=True)
    elif option =="alexnet":
        return models.alexnet(pretrained=True)
    elif option == "vgg19":
        return models.vgg19(pretrained=True)
    elif option=="resnet50":
        return models.resnet50(pretrained=True)
    elif option=="googlenet":
        return models.googlenet(pretrained=True)
    elif option=="squeezenet1_1":
        return models.squeezenet1_1(pretrained=True)
    else:
        return None

In [ ]:
def train_loader(b,path_train):

  train_data = TripletData(path_train, train_transforms,"train",2)
  train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=b, shuffle=True, num_workers=4)
  return train_loader

In [ ]:
def extraction(model,PATH_TRAIN,choix):
 if choix== "Canberra":
    faiss_index= faiss.IndexFlat(1000,faiss.METRIC_Canberra)
 if choix== "BrayCurtis":
    faiss_index= faiss.IndexFlat(1000,faiss.METRIC_BrayCurtis)
 if choix=="L1":
    faiss_index= faiss.IndexFlat(1000,faiss.METRIC_L1)
 if choix=="L2":
    faiss_index = faiss.IndexFlatL2(1000)
 model.eval()
 im_indices = []
 with torch.no_grad():
    for f in glob.glob(os.path.join(PATH_TRAIN, '*/*')):
        im = Image.open(f).convert("RGB")
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()

        preds = model(im)
        preds = np.array([preds[0].cpu().numpy()])
        faiss_index.add(preds) #add the representation to index
        im_indices.append(f)
 return im_indices , faiss_index

In [ ]:
# Preprocess the image
def preprocess(image, size=224):
    transform = T.Compose([
        T.Resize((size,size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(image)

In [ ]:
def Averageprecision(code, classe,indices,df,csvtrain):
            correctImages = []
            SimImg=0
            compRes=0
            for index in indices:
              idimage=os.path.splitext(os.path.basename(df.iloc[index][1]))[0]

              classesimilaire=csvtrain.at[int(idimage),"05_class"]
              codesimil=csvtrain.at[int(idimage),"irma_code"]
              codesimil=codesimil.split('-')
              codesimil=codesimil[2]

              print("la classe IMG req",classe ,"les code",code,"classe similaire or not est", classesimilaire,"le code",codesimil)
              #print("la classe 6 de limage similaire or not est", classesimilaire1)
              compRes=compRes+1
              if classe==classesimilaire or code==codesimil :
                 SimImg=SimImg+1
                 num =(SimImg)/compRes
                 correctImages.append(num)
            if SimImg==0:
                 SimImg=1
            return sum(correctImages)/SimImg

In [ ]:
def dezipdata(file):

#dans le cas ( file is umpty)
   if file.filename == '':
    flash('No selected file')
    return redirect(url_for('hello'))
#dans le cas (file exist)
   if file :
     dirpath="/content/drive/MyDrive/color/coul/static/data"
     shutil.rmtree(dirpath)
     os.mkdir(dirpath)
     filename = secure_filename(file.filename)
     pathfile=os.path.join('/content', filename)
     print("pathfile de data",pathfile)
     file.save(pathfile)
     patoolib.extract_archive(pathfile,outdir="/content/drive/MyDrive/color/coul/static/data/")
     print("dezip done")
     os.remove(pathfile)
     print("remove done")
def uploadimage(file):
   filename = secure_filename(file.filename)
   print(filename)
   imagepath=os.path.join('/content/drive/MyDrive/color/coul/static/imagereq/', filename)
   print(imagepath)
   file.save(imagepath)
   return imagepath

def compteur_images(list):
      number_files = len(list)
      print("numbeeeer",number_files)
      print(list)
      compteur=0
      for file in  list :
        print('le dossier ',file)
        dossier=os.listdir('/content/drive/MyDrive/color/coul/static/data/'+file)
        print("le contenu du dossier ", dossier)
        for item in dossier:
          print("l'image" ,item)
          compteur=compteur+1
      print("nombre d'images dans la base" ,compteur)
      return compteur

In [ ]:
def train_model(model,op,lr,bs,ep):
   modelname=model
   model=get_model(model).cuda()
   optimizer=get_optimizer(model,op,float(lr))
   device = 'cuda'
   triplet_loss = TripletLoss()
   path="/content/drive/MyDrive/color/coul/static/data"
   traindata=train_loader(int(bs),path)

#début d'entrainement
   os.chdir("/content")
   for file in glob.glob("*.pt"):
    path="/content/"+file
    os.remove(file)

   for epoch in range(int(ep)):
    model.train()
    epoch_loss = 0.0
    for data in tqdm(traindata):
        optimizer.zero_grad()
        x1,x2,x3 = data
        e1 = model(x1.to(device))
        e2 = model(x2.to(device))
        e3 = model(x3.to(device))
        loss = triplet_loss(e1,e2,e3)
        epoch_loss += loss
        loss.backward()
        optimizer.step()
        print("Train Loss: {}".format(epoch_loss.item()))
        torch.save(model.state_dict(), modelname+".pt")


In [ ]:
def saveFaiss(faiss_index):
  chunk = faiss.serialize_index(faiss_index)
  np.save("index.npy", chunk)

In [ ]:
def occurance (modele):
 pathmodFeat="/content/drive/MyDrive/color/coul/static/modFeat/"
 list=[]
 lesnomsdossier=os.listdir(pathmodFeat)
 for nomdossier in lesnomsdossier:
     path=pathmodFeat+str(nomdossier)
     os.chdir(path)
     file = glob.glob("*")
     x=file.count(modele)
     if x==1:
       return path




In [ ]:
import os
from flask import *
from flask_ngrok import run_with_ngrok
import pickle
import numpy as np
import patoolib
import shutil
import faiss
from werkzeug.utils import secure_filename
app = Flask(__name__,template_folder='/content/drive/MyDrive/color/coul/template',static_folder='/content/drive/MyDrive/color/coul/static',static_url_path='')
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/evaluation")
def evaluation():
  list=[]
#recupération de la liste des modèles
  pathmodFeat="/content/drive/MyDrive/color/coul/static/modFeat/"
  lesnomsdossier=os.listdir(pathmodFeat)
  for nomdossier in lesnomsdossier:
     path=pathmodFeat+str(nomdossier)
     os.chdir(path)
     for file in glob.glob("*.pt"):
         list.append(file)

  return render_template('evaluation.html',list=list)

@app.route("/extract")
def extract():
   return render_template('extract.html')

@app.route("/search")
def search():
  list=[]
#recupération de la liste des modèles
  pathmodFeat="/content/drive/MyDrive/color/coul/static/modFeat/"
  lesnomsdossier=os.listdir(pathmodFeat)
  for nomdossier in lesnomsdossier:
     path=pathmodFeat+str(nomdossier)
     os.chdir(path)
     for file in glob.glob("*.pt"):
         list.append(file)

  return render_template('search.html', list=list)

@app.route("/")
def hello():
   return render_template('advanced_form_components1.html')

@app.route('/', methods = [ 'POST'])
def upload_file():
#dezip data
    file = request.files['file']
    dezipdata(file)
 #compteur d'image de la base
    list = os.listdir('/content/drive/MyDrive/color/coul/static/data') # dir is your directory path
    message="data set succesfuly uploaded with "+str(compteur_images(list))+" images"
    flash(message,'upload')
    return redirect(url_for('hello'))

@app.route('/train', methods = ['POST'])
def get_param():

#recupération des hyperparamètres de l'entrainement
   op=request.form.get("op")
   print(op)
   lr=request.form.get("lr")
   print(lr)
   ep=request.form.get("ep")
   print(ep)
   bs=request.form.get("b")
   print(bs)
   model=request.form.get("model")
   print(model)
   print("traaain ...")
   train_model(model,op,lr,bs,ep)
   flash (u'The model training is done','train' )
   return redirect(url_for('hello'))

@app.route('/ex', methods = ['POST'])
def extrFeat():
  path="/content/drive/MyDrive/color/coul/static/data"
  pathmodFeat="/content/drive/MyDrive/color/coul/static/modFeat/"
  co=0
  list=[]
  src_file=[]
  device = 'cuda'
#avoir le path du modele,
  os.chdir("/content")
  file=glob.glob("*.pt")
  modelname= file[0].split(".pt")
  modelname=modelname[0]
  encoderpath= os.path.join("/content",file[0])
# création d'un nouveau folder pour modele et faiss
  os.chdir(pathmodFeat)
  for file in glob.glob("*"):
   list.append(file)
  if not list:
   list.append(0)
  num=int(max(list))
  numfile=num+1
  newpath='/content/drive/MyDrive/color/coul/static/modFeat/'+str(numfile)
  print("le new path",newpath)
  os.mkdir(newpath)
#load du modele
  print("loaaad...")
  model=get_model(modelname).cuda()
  model.load_state_dict(torch.load(encoderpath, map_location=device))
#extraction
  with torch.no_grad():
   choix=request.form.get("metric")
   print ("extraction features...")
   result=extraction(model,path,choix)
   im_indices = result[0]
   faiss_index=result[1]
   print("les paths ",im_indices)
   print("les features with index ",faiss_index)
   print ("extraction is done")

#save im_indices in the newpath folder
   df = pd.DataFrame(im_indices)
   csvpath=newpath+'/indicesirma.csv'
   df.to_csv(csvpath)

#save faiss in content
   os.chdir('/content')
   saveFaiss(faiss_index)

#deplacement du faiss et modèle au new folder
  faisspath= "/content/index.npy"
  src_file.append(encoderpath)
  src_file.append(faisspath)
  for filepath in src_file:
   if os.path.isfile(filepath):
    shutil.move(filepath,newpath)
  flash (u'Features extraction is done, Now click on image search button on your left to continue','extr' )
  return redirect(url_for('hello'))


@app.route('/rechIMG', methods = ['POST'])
def searIMG():
 csvtrain=pd.read_csv("/content/drive/MyDrive/color/coul/static/csv/ImageCLEFmed2009_train_codes.02.csv",sep=';')
 csvtrain=csvtrain.set_index("image_id",drop=False)
 csvtest=pd.read_csv("/content/drive/MyDrive/color/coul/static/csv/ImageCLEFmed2009_test_codes.03.csv",sep=';')
 csvtest=csvtest.set_index("image_id",drop=False)
 device='cuda'
#uploader image requete dans le folder imagereq
 file = request.files['file']
 imgfolder="/content/drive/MyDrive/color/coul/static/imagereq"
 shutil.rmtree(imgfolder)
 os.mkdir(imgfolder)
 imagepath=uploadimage(file)
#recuperer le nombre des images
 n=request.form.get("number")
 print("number ",n)

#get the im_indices and faiss after getting name model
 m=request.form.get("model")
 print("modele ",m)
 pathvoulu=occurance(m)
 pathcsv=pathvoulu+'/indicesirma.csv'
 pathfaiss=pathvoulu+'/index.npy'
 #pathcsv="/content/drive/MyDrive/IRMA/indicerasnetmin.csv"
 #pathfaiss="/content/drive/MyDrive/IRMA/indexresnetmin.npy"
 encoderpath=pathvoulu+'/'+m
 print(pathcsv)
 print(pathfaiss)

#load du modele
 print("loaaad...")
 modelname = m.split(".pt")
 print(modelname[0])
 model=get_model(modelname[0]).cuda()
 model.load_state_dict(torch.load(encoderpath, map_location=device))

#transorm limage requete get id des images similaires
 faiss_index = faiss.deserialize_index(np.load(pathfaiss))
 with torch.no_grad():
  im = Image.open(imagepath).convert("RGB")
  im = im.resize((224,224))
  im = torch.tensor([val_transforms(im).numpy()]).cuda()
  test_embed = model(im).cpu().numpy()
  _, I = faiss_index.search(test_embed, int(n))
  index = I.tolist()
  print(index)
  indices = index[0]
  print("les indices ",indices)
#put the simialaire images in folderSIM
  df=pd.read_csv(pathcsv)
  pathsim="/content/drive/MyDrive/color/coul/static/FolderSIM"
  shutil.rmtree(pathsim)
  os.mkdir('/content/drive/MyDrive/color/coul/static/FolderSIM')
  cm=0
  k=0
  idimaget=os.path.splitext(os.path.basename(imagepath))[0]
  print("id image testtt",idimaget)
  classe=csvtest.at[int(idimaget),"05_class"]
  for index in indices:
   pathImage=df.iloc[index][1]
   print(pathImage)
   idimage=os.path.splitext(os.path.basename(pathImage))[0]
   print("id image traiin",idimage)
   classesimilaire=csvtrain.at[int(idimage),"05_class"]
   im = Image.open(pathImage).convert("RGB")
   if classesimilaire==classe:
    bimg =  ImageOps.expand(im, 15, '#7BFB3F')
   else :
    bimg =  ImageOps.expand(im, 15, '#ff1919')
   k=k+1
   pathjdid=pathsim+'/'+str(k)+'.png'
   print("voila path jdid", pathjdid)
   bimg.save(pathjdid)
   #print(pathImage)
   #shutil.copy(pathImage,pathsim)
   cm=cm+1
   print("dkhal mara",cm)
 #dossier saliency
 pathSal='/content/drive/MyDrive/color/coul/static/saliencyR'
 shutil.rmtree(pathSal)
 os.mkdir(pathSal)

#dossier grad
 pathGRAD='/content/drive/MyDrive/color/coul/static/GradCam'
 shutil.rmtree(pathGRAD)
 os.mkdir(pathGRAD)

#étape saliency
 i=0
 model = get_model1(modelname[0]).eval()
 List=os.listdir('/content/drive/MyDrive/color/coul/static/FolderSIM')
 for imge in List:
  i=i+1
  print(imge)
  img = Image.open('/content/drive/MyDrive/color/coul/static/FolderSIM/'+imge).convert("RGB")
  X = preprocess(img)
  model.eval()
  X.requires_grad_()
  scores = model(X)
  score_max_index = scores.argmax()
  score_max = scores[0,score_max_index]
  score_max.backward()
  saliency, _ = torch.max(X.grad.data.abs(),dim=1)
  plt.imshow(saliency[0], cmap=plt.cm.hot)
  plt.axis('off')
  print("saving...")
  pathS=pathSal+"/"+str(i)
  plt.savefig(pathS)
#étape Grad Cam

  cam_extractor = LayerCAM(model, 'layer4', 'fc')
  img = torchvision.io.read_image('/content/drive/MyDrive/color/coul/static/FolderSIM/'+imge, mode=torchvision.io.image.ImageReadMode.RGB)
  input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  out = model(input_tensor.unsqueeze(0))
  activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
  result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
  plt.imshow(result)
  plt.axis('off')
  plt.tight_layout()
  print("saving...")
  pathG=pathGRAD+"/"+str(i)
  plt.savefig(pathG)
 return redirect(url_for('show_index'))

@app.route('/show')
def show_index():
#affichage image requete
    imgreq=os.listdir('/content/drive/MyDrive/color/coul/static/imagereq')
    for img in imgreq:
       imgreq=img
    imgreq='imagereq/'+imgreq
    print("image req",imgreq )

#affichage images similaires
    imageList=os.listdir('/content/drive/MyDrive/color/coul/static/FolderSIM')
    imageList=['FolderSIM/'+ image for image in imageList]
    print("path ",imageList)

#affichage des image Saliency
    ImagesSal=os.listdir('/content/drive/MyDrive/color/coul/static/saliencyR')
    ImagesSal=['saliencyR/'+ image for image in ImagesSal]
    print("path ",ImagesSal)

#affichage des image GradCam
    ImageGRAD=os.listdir('/content/drive/MyDrive/color/coul/static/GradCam')
    ImageGRAD=['GradCam/'+ image for image in ImageGRAD]
    print("path ",ImageGRAD)

    return render_template("search1.html",user_image =imageList,imgreq=imgreq, ImagesSa=ImagesSal,ImageGRAD=ImageGRAD)

@app.route('/test', methods = ['POST'])
def upload_file_test():

#upload du dossier test et les csv  dans la page d'evaluation dans content
   files = request.files.getlist("file")
   for file in files:
      file.save(os.path.join('/content', file.filename))

#création du dossier 'dataset' pour le depot des images test
   imgfolder="/content/drive/MyDrive/color/coul/static/Datatest/"
   shutil.rmtree(imgfolder)
   os.mkdir(imgfolder)

#déplacement du dossier test from content to dataset
   os.chdir("/content/")
   datarar=glob.glob("*.rar")
   pathrar='/content/'+datarar[0]
   print ("path of the rar file",pathrar)
   patoolib.extract_archive(pathrar,outdir="/content/drive/MyDrive/color/coul/static/Datatest/")
   os.remove(pathrar)

#création du dossier csv et déplacement des fichiers csv from content to dossier csv
   imgfolder="/content/drive/MyDrive/color/coul/static/csv"
   shutil.rmtree(imgfolder)
   os.mkdir(imgfolder)

   filescsv=glob.glob("*.csv")
   print("esm csv",filescsv)
   src='/content/'
   for file_name in filescsv:
    full_file_name = os.path.join(src, file_name)
    print(full_file_name)
    if os.path.isfile(full_file_name):
      shutil.copy(full_file_name, '/content/drive/MyDrive/color/coul/static/csv/')
      print("copyy")
      os.remove(full_file_name)
   flash(u'file succesfuly uploaded','upload')
   return redirect(url_for('evaluation'))


@app.route('/evaluation', methods = ['GET', 'POST'])
def map():
   legend=r'Images Average Precision %'
   pathlist=[]
   aplist=[]
   path ="/content/drive/MyDrive/color/coul/static/Datatest/testdata"
   device='cuda'

#get the im_indices and faiss after getting name model
   m=request.form.get("model")
   print("modele ",m)
   pathvoulu=occurance(m)
   pathfaiss=pathvoulu+'/index.npy'
   pathcsv=pathvoulu+'/indicesirma.csv'
   encoderpath=pathvoulu+'/'+m
   index = faiss.deserialize_index(np.load(pathfaiss))
   print(pathfaiss)
   print(encoderpath)

#load du modele
   print("loaaad...")
   modelname = m.split(".pt")
   print(modelname[0])
   model=get_model(modelname[0]).cuda()
   model.load_state_dict(torch.load(encoderpath, map_location=device))
#les csv
   df=pd.read_csv(pathcsv)
   csvtrain=pd.read_csv("/content/drive/MyDrive/color/coul/static/csv/ImageCLEFmed2009_train_codes.02.csv",sep=';')
   csvtrain=csvtrain.set_index("image_id",drop=False)
   csvtest=pd.read_csv("/content/drive/MyDrive/color/coul/static/csv/ImageCLEFmed2009_test_codes.03.csv",sep=';')

#parcourir les images du dossier test et calculer l AP de chaque image
   csvtest=csvtest.set_index("image_id",drop=False)
   with torch.no_grad():
    number=0
    for f in os.listdir(path):
     number=number+1
     print("path of the test photo", number ," ",f)
     im = Image.open(os.path.join(path,f)).convert("RGB")
     im = im.resize((224,224))
     im = torch.tensor([val_transforms(im).numpy()]).cuda()
     test_embed = model(im).cpu().numpy()
     _, I = index.search(test_embed, 10)
     indices_list = I.tolist()
     indices = indices_list[0]
     idimage=os.path.splitext(f)[0]
     pathlist.append(idimage)

     print("the path",f,"idimage",idimage)
     classe=csvtest.at[int(idimage),"05_class"]
     code=csvtest.at[int(idimage),"irma_code"]
     code=code.split('-')
     code=code[2]
     ap = Averageprecision(code,classe,indices,df,csvtrain)
     ap=ap*100
     aplist.append(ap)
     map=sum(aplist)/len(aplist)
   return render_template('evaluation1.html',values=aplist, labels=pathlist, legend=legend, map=map)


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'module' object is not callable


 * Running on http://46b7-34-126-91-48.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:44] "GET /css/style-responsive.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:44] "GET /img/covid.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:44] "GET /lib/bootstrap-datetimepicker/datertimepicker.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:44] "GET /css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:45] "GET /lib/jquery.nicescroll.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:45] "GET /lib/advanced-form-components.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:45] "GET /lib/jquery.dcjqaccordion.2.7.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:45] "GET /lib/jquery-ui-1.9.2.custom.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 20:04:45] "GET /lib/jquery.scrollTo.min.js HTTP/1.1" 200 -
INFO:werkzeug:127